## Data Cleaning Plan

We have three data sets:
- Markers' bios and metadata (markers_bios)
- Followers' bios and metadata (followers_bios)
- All brands and their followers (markers-followers)


Step by step plan:
1. Load the bios of followers, and the marker-follower file. 
    - Provide summary statistics of users and brands. How many brands do we have? How many followers? Any missing data, duplicates etc.?

2. Filter on marker-follower df:
    - Create a dictionary of counts brands per follower
    - Remove users that follow less than 5 (or more) brands
    - Continuously track numbers of users removed
    - Match the Follower_Ids in the now filtered marker-follower df with the follower-bio df. As such, the follower bios will only include users that follow more than five brands. Subsequent filters will be on the correct users (up to date follower-bios).

3. Do the filters on the follower-bios:
    - Remove users with less than 25 followers
    - Remove users with less than 100 tweets

4. Filter based on language: keep only french accounts









In [1]:
# Standard library imports
from collections import defaultdict
from datetime import datetime
import csv
import html
import os
import re
import sys

# Third-party library imports
import dask.dataframe as dd
from joblib import Parallel, delayed
from langdetect import detect_langs, LangDetectException, DetectorFactory
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import psutil
import pickle
import regex
import seaborn as sns
from unidecode import unidecode
import gcld3
import ftfy
import importlib

# Local application/library specific imports
import utils2
from utils2 import *

## 1. Load files and summary stats

In [2]:
# # Load the data files and rename ID columns
importlib.reload(utils2)

# Load markers-followers
load_path = '/home/livtollanes/SocialMarkers'
file = 'markers_followers_2023-05-19.csv'

req_cols = ['id', 'follower_id']
dtypes = {'id': 'object',
          'follower_id': 'object'}

markers_followers = utils2.fileloader(load_path, file, req_cols, dtypes)


#rename the twittwer id column to follower id 
markers_followers.rename(columns={'id':'marker_id'}, inplace=True)

In [3]:
# Load the followers bios and rename ID columns
load_path = '/home/livtollanes/SocialMarkers'
file = 'markers_followers_bios_2023-05-19.csv'

req_cols = ['twitter_id', 'id', 'screen_name', 'description', 'location', 'tweets', 'followers', 'friends', 'likes', 'lists','timestamp_utc']

dtypes = {
    'twitter_id': 'object',
    'id': 'object',
    'screen_name': 'object',
    'description': 'object',
    'location': 'object',
    'tweets': 'float64',
    'followers': 'float64',
    'friends': 'float64',
    'witheld_in_countries': 'float64'
}

followers_bios = utils2.fileloader(load_path, file, req_cols, dtypes)



#rename the twittwer id column to follower id 
followers_bios.rename(columns={'twitter_id':'follower_id'}, inplace=True)

Summary statistics

In [4]:
#utils2.summary_stats(followers_bios, print_dtypes=False)

Shape of DataFrame:  (70666646, 11)

Columns in DataFrame:  ['follower_id', 'id', 'screen_name', 'description', 'timestamp_utc', 'location', 'tweets', 'followers', 'friends', 'likes', 'lists']

Number of unique values in 'follower_id':  70666646
Number of duplicate values in 'follower_id':  0

Number of unique values in 'id':  70642661
Number of duplicate values in 'id':  23984
Number of missing values in 'follower_id':  0
Number of missing values in 'id':  23985
Number of missing values in 'screen_name':  23986
Number of missing values in 'description':  42027215
Number of missing values in 'timestamp_utc':  23985
Number of missing values in 'location':  47956041
Number of missing values in 'tweets':  23985
Number of missing values in 'followers':  23985
Number of missing values in 'friends':  23985
Number of missing values in 'likes':  23985
Number of missing values in 'lists':  23985

Number of duplicate rows:  0


In [5]:
# importlib.reload(utils2)
# utils2.summary_stats(markers_followers, print_dtypes=False)

Shape of DataFrame:  (126345412, 2)

Columns in DataFrame:  ['marker_id', 'follower_id']

Number of unique values in 'follower_id':  70636295
Number of duplicate values in 'follower_id':  55709117

Number of unique values in 'marker_id':  236
Number of duplicate values in 'marker_id':  126345176

Number of missing values in each column:
'marker_id':  0
'follower_id':  0

Number of duplicate rows:  2357493


In [5]:
# Sort the DataFrame to ensure that duplicates are next to each other
markers_followers_sorted = markers_followers.sort_values(by=list(markers_followers.columns))

# Find duplicates in the sorted DataFrame
duplicates = markers_followers_sorted[markers_followers_sorted.duplicated(keep=False)]

# Print the first 10 rows of duplicates (5 pairs)
print(duplicates.head(10))

         marker_id          follower_id
81032594  25053299            100000025
89256298  25053299            100000025
79323950  25053299  1000001004220420096
87547795  25053299  1000001004220420096
79369517  25053299  1000001771266232320
87593356  25053299  1000001771266232320
79562687  25053299           1000001815
87786516  25053299           1000001815
79371019  25053299  1000002790238777352
87594858  25053299  1000002790238777352


In [6]:
#drop the duplicates in markers_followers
markers_followers.drop_duplicates(keep='first', inplace=True)

In [7]:
markers_followers.shape

(123987919, 2)

In [8]:
compare_column_values(followers_bios, markers_followers, 'follower_id')

There are 30351 unique values in df1 that don't exist in df2.
There are 0 unique values in df2 that don't exist in df1.


## 2. Filter the marker-follower df

- Filter the marker-follower df:
    - Remove users that follow less than 5 (or more) brands

    - Continuously track numbers of users removed
    
    - Match the Follower_Ids in the now filtered marker-follower df with the follower-bio df. As such, the follower bios 
    will only include users that follow more than five brands. Subsequent filters will be on the correct users (up to date follower-bios).

Remove users that follow less than 5 brands

In [9]:
n = 5  # minimal number of brands followed required to be included in the analysis
importlib.reload(utils2)
markers_followers_5 = utils2.filter_followers(markers_followers, 'follower_id', n)

66693204 followers follow less than 5 brands (94.42% of the total followers).
After removing these followers, 3943091 followers are left (5.58% of the followers in the inputted df).


In [12]:
#number of unique brands left

# Get the unique marker_id values in the original and filtered DataFrames
original_brands = set(markers_followers['marker_id'].unique())
filtered_brands = set(markers_followers_5['marker_id'].unique())

# Find the brands that are in the original DataFrame but not in the filtered DataFrame
removed_brands = original_brands - filtered_brands

# Print the removed brands
print("Removed brands:", removed_brands) #corresponds to "Napapijiri97", which kindof sounds like a fake profile


Removed brands: {'1059975643'}


Match the IDs in the filtered marker-follower df with the follower bio df, so that the follower bios only are for those who follow at least 5 brands

In [13]:
followers_bios_5 = utils2.streamline_IDs(source = markers_followers_5, df_tofilter= followers_bios, 'follower_id')

Number of unique follower_id in source DataFrame: 3943091
Number of unique follower_id in filtered DataFrame after filtering: 3943091
Removed 66723555 rows from the DataFrame to be filtered.
3943091 rows are left in the filtered DataFrame.


In [14]:
compare_column_values(followers_bios_5, markers_followers_5, 'follower_id')   

There are 0 unique values in df1 that don't exist in df2.
There are 0 unique values in df2 that don't exist in df1.


## 3. Do the filters on the follower-bios:
- Remove users with less than 25 followers
- Remove users with less than 100 tweets
- Update the markers-followers df to match the now filtered bio df
- Filter based on language: keep only french accounts


In [15]:

followers_bios_fullfilter = utils2.filter_by_tweets_and_followers(followers_bios_5, min_followers= 25, min_tweets= 100)


Removed 2750559 rows.
1192532 rows are left.


Again, remove the follower_Ids in markers-followers that don't occur in the newly filtered  followers_bios_nd5_tweets_followers

In [16]:
markers_followers_fullfilter = utils2.streamline_IDs(source= followers_bios_fullfilter, df_tofilter=markers_followers_5, column='follower_id')

Number of unique follower_id in source DataFrame: 1192532
Number of unique follower_id in filtered DataFrame after filtering: 1192532
Removed 18992709 rows from the DataFrame to be filtered.
9614122 rows are left in the filtered DataFrame.


In [17]:
compare_column_values(followers_bios_fullfilter, markers_followers_fullfilter , 'follower_id')

There are 0 unique values in df1 that don't exist in df2.
There are 0 unique values in df2 that don't exist in df1.


In [19]:
#Before writing ti csv, clean description column to avoid writing problems
importlib.reload(utils2)

followers_bios_fullfilter = utils2.process_description(followers_bios_fullfilter, 'description')

In [21]:
summary_stats(markers_followers_fullfilter, print_dtypes=False)

Shape of DataFrame:  (9614122, 2)

Columns in DataFrame:  ['marker_id', 'follower_id']



Number of unique values in 'follower_id':  1192532
Number of duplicate values in 'follower_id':  8421590

Number of unique values in 'marker_id':  235
Number of duplicate values in 'marker_id':  9613887

Number of missing values in each column:
'marker_id':  0
'follower_id':  0

Number of duplicate rows:  0


In [22]:
# # #Now write the two dfs to csvs to save them in case something happens
markers_followers_fullfilter.to_csv('/home/livtollanes/NewData/markers_followers_cleaned_nolang.csv', encoding='utf-8', index=False)

followers_bios_fullfilter.to_csv('/home/livtollanes/NewData/followers_bios_cleaned_nolang3.csv', sep=',', encoding='utf-8', index=False, quoting=csv.QUOTE_NONNUMERIC)



## 4. Filter based on language: keep only french accounts
- Use language recognition alorithms to filter the follower_bios. 
- We only want french language bios to be included


In [ ]:
#Load marker followers
full_path1 = '/home/livtollanes/NewData/markers_followers_cleaned_nolang.csv'
req_cols = ['marker_id', 'follower_id']
dtypes = {'marker_id': 'object',
          'follower_id': 'object'}

markers_followers_clean = pd.read_csv(full_path1, encoding='utf-8', dtype=dtypes, usecols=req_cols)

In [ ]:
# #Loading the followers bios (with cleaned description column)
full_path = '/home/livtollanes/NewData/followers_bios_cleaned_nolang3.csv'

req_cols = ['follower_id', 'screen_name', 'description', 'description_cleantext', 'location', 'tweets', 'followers', 'friends', 'likes', 'lists','timestamp_utc']

dtypes = {
    'follower_id': 'object',
    'screen_name': 'object',
    'description': 'object',
    'description_cleantext': 'object',
    'location': 'object',
    'tweets': 'float64',
    'followers': 'float64',
    'friends': 'float64'
}

follower_bios_cleaned3 = pd.read_csv(full_path, usecols=req_cols, dtype=dtypes, engine= 'python')

In [ ]:
compare_column_values(follower_bios_cleaned3, markers_followers_clean, 'follower_id')

#The follower_ids are still streamlined, indicating that writing and reading of the cleaned dfs was successful

In [ ]:
#should probably do summary stats on the dfs again, to make sure that no strange things have happened during the cleaning process
summary_stats(follower_bios_cleaned3, print_dtypes= True)

In [ ]:
summary_stats(markers_followers_clean, print_dtypes= True)

Here comes the language detection

- First language detection is with the langdetect package. This package is based on Google's language detection API. On medium, it is actually reported that this package may not perform very accurate on short or mixed lnaguage texts, which is exactly what we are deailing with in twitter bios. 
- Gcld3 mmight be better at handling short text inputs like bios. By reading the documentation, gcld3 seems to be more fitting for social media data language detection

In [ ]:
importlib.reload(utils2)
# Create a copy of the DataFrame for each function
follower_bios_cleaned3_copy1 = follower_bios_cleaned3.copy()
follower_bios_cleaned3_copy2 = follower_bios_cleaned3.copy()

# Use the copied DataFrames in the functions
lang = utils2.add_and_detect_language(follower_bios_cleaned3_copy1, 'description_cleantext', seed = 3, n_jobs=-1)
lang2 = utils2.detect_language_gcld3(follower_bios_cleaned3_copy2, 'description_cleantext', n_jobs=-1)

In [ ]:
total_rows = lang.shape[0]

french_percent = (lang[lang['language'] == 'fr'].shape[0] / total_rows) * 100
english_percent = (lang[lang['language'] == 'en'].shape[0] / total_rows) * 100
unknown_percent = (lang[lang['language'] == 'unknown'].shape[0] / total_rows) * 100
other_percent = 100 - french_percent - english_percent - unknown_percent

print("French: ", french_percent, "%")
print("English: ", english_percent, "%")
print("Unknown: ", unknown_percent, "%")
print("Other: ", other_percent, "%")

nan_percent = (lang['description_cleantext'].isna().sum() / total_rows) * 100
print("NaN in description_cleantext: ", nan_percent, "%")

In [ ]:
total_rows = lang2.shape[0]

french_percent = (lang2[lang2['language'] == 'fr'].shape[0] / total_rows) * 100
english_percent = (lang2[lang2['language'] == 'en'].shape[0] / total_rows) * 100
unknown_percent = (lang2[lang2['language'] == 'unknown'].shape[0] / total_rows) * 100
other_percent = 100 - french_percent - english_percent - unknown_percent

print("French: ", french_percent, "%")
print("English: ", english_percent, "%")
print("Unknown: ", unknown_percent, "%")
print("Other: ", other_percent, "%")

nan_percent = (lang2['description_cleantext'].isna().sum() / total_rows) * 100
print("NaN in description_cleantext: ", nan_percent, "%")

### Manually creating test sets to compare the two language detection methods

For the langdetect df

In [ ]:
test1 = lang.sample(n=100, random_state=1)
test2 = lang2.sample(n=100, random_state=1)

In [ ]:
test1.head(5)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
test1[['screen_name', 'location','description_cleantext', 'language']]

In [ ]:


true_labels = {
    'MorganJewelers1': 'non_fr',
    'LeighAnnTowne': 'non_fr',
    'ALBrutel': 'NA',
    'Fonkwadiour1': 'non_fr',
    'JUSTINAGROSSO': 'NA',
    'SarahRadif': 'non_fr',
    'CarineMayol': 'NA',
    'ManonMondou': 'fr',
    'erickleangar': 'NA',
    'elaineguimarae': 'non_fr',
    'danyunes': 'fr',
    'kuronapilled': 'non_fr',
    'KiriYaji': 'non_fr',
    'floriannisolle': 'fr',
    'Naman93726237': 'fr', #french but from guinea
    'doktor_vananga': 'non_fr', 
    'Slaiidh': "non_fr", #wrote non. could be another language
    'StarRaull': "non_fr",
    'SPeperstraete': 'non_fr',
    'yorgaine_lyon69': 'fr',
    'blinameta': 'non_fr',
    'AnnaRus75': 'non_fr',
    'BernTurner': 'NA',
    'lauriebeebe2': 'non_fr',
    'JeremyyPalazzo': 'NA',
    'GalerieCoulange': 'fr',
    'HaziqkaJ': 'NA',
    'DaraCheick': 'non_fr', #but seems to belong to a french uni
    'mmdlbdrny2': 'NA',
    'blandineleonie': 'fr',
    'CarlosO324': 'non_fr',
    'joshuatoney3': 'non_fr',
    'StephaneWayler': 'fr',
    'JessieV_214': 'non_fr',
    'Apartofsuzanne': 'non_fr',#but writes that she is french, and location is Paris
    'katuxka1': 'NA',
    'tamaitanya': 'NA',
    'sulagnasays': 'non_fr',
    'nicolesimon': 'non_fr',
    'lunasitah': 'non_fr',
    'ln971': 'fr',
    'olive_cas': 'non_fr',
    'Mercedeszatfc': 'NA',
    'hady_saad': 'non_fr',
    'GatienLeroux': 'fr',
    'Meluxiam': 'non_fr',
    'yulimadera10': 'non_fr',
    'RyanBrossault': 'fr',
    'FlotillaMRY': 'non_fr',
    'love98_daniela': 'non_fr',
    'jucag4115': 'NA',
    'Bxwiz': 'non_fr',
    'MLECOMTE': 'fr',
    'DelgadoJanahi': 'non_fr',
    'f1rmin': 'fr',
    'SylvainWalczak': 'NA',
    'SoyLigiawn': 'non_fr',
    'javielo26': 'non_fr',
    'CherrybangMUA': 'non_fr',
    '1927_albert': 'non_fr',
    'aayyoud': 'NA',
    'princessivy28': 'non_fr',
    'drumz420': 'NA',
    'Cjruin': 'non_fr',
    'xXallymayXx': 'non_fr',
    'NaAutacaace': 'fr',
    'RosaLuc15360022': 'non_fr',
    'PatrikWinston': 'non_fr',
    'mwa_sisqo': 'NA',
    'vankrug': 'NA',
    'miktoi': 'non_fr',
    'cescoeco': 'non_fr',
    'floo_ncy': 'NA',
    'Quentin_1411': 'non_fr',
    'Tvy_Tk': 'NA',
    'AlbanLeneveu': 'NA',
    'adh2311': 'non_fr',
    'loaizayose': 'NA',
    'mathilde_Fparis': 'fr',
    'gouillardmichae': 'fr',
    'RomainSprynski': 'fr',
    'styledscience': 'non_fr',
    'amhammadi': 'NA',
    'AliRazaSharif2': 'NA',
    'JahanLutz': 'fr',
    'HERVEJEHL': 'fr',
    'CatherineMSch': 'fr',
    'PoliKeyCo': 'non_fr',
    'HisGraceth': 'non_fr',
    'verstagoat': 'non_fr',
    'G_deLinares': 'fr', 
    'Granadechkou': 'fr',
    'MommaDandine': 'NA',
    'allionesolution': 'non_fr',
    'Ross75016': 'fr',
    'DorignyTheo': 'NA',
    'nikkolasg1': 'non_fr',
    'InesNau': 'fr',
    'FrenchEmperior': 'non_fr',
    'BekaEssick': 'non_fr'}

In [ ]:
def label_language(lang):
    if lang == 'unknown':
        return 'NA'
    elif lang == 'fr':
        return 'fr'
    else:
        return 'non_fr'

test1['pred_lang'] = test1['language'].apply(label_language)
test2['pred_lang'] = test2['language'].apply(label_language)

test1['true_lang'] = test1['screen_name'].map(true_labels)
test2['true_lang'] = test2['screen_name'].map(true_labels)

In [ ]:
test1.head(5)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Calculate metrics for test1
print("Metrics for test1:")
print(classification_report(test1['true_lang'], test1['pred_lang']))
print("Accuracy:", accuracy_score(test1['true_lang'], test1['pred_lang']))

# Calculate metrics for test2
print("\nMetrics for test2:")
print(classification_report(test2['true_lang'], test2['pred_lang']))
print("Accuracy:", accuracy_score(test2['true_lang'], test2['pred_lang']))

- Still lack the actual language filtering
- Think of a more sophisticated way to locate french users than only using language. This method should capture french users that write in english, and in french, and avoid selecting users that are writing in french but are not actually french. 